# The Property Framework

In [1]:
import numpy as np

Since version 0.2.0 Qiskit Nature includes the _Property_ framework. This framework replaces the legacy driver return types like `QMolecule` and `WatsonHamiltonian` with a more modular and extensible approach.

In this tutorial, we will walk through the framework, explain its most important components and show you, how you can extend it with a custom _property_ yourself.

## What is a `Property`?

At its core, a `Property` is an object complementing some raw data with functions that allow you to transform/manipulate/interpret this raw data. This "definition" is kept rather abstract on purpose, but what it means is essentially the following.
A `Property`:
* represents a physical observable (that's the raw data)
* can be expressed as an operator
* can be evaluated with a wavefunction
* provides an `interpret` method which gives meaning to the eigenvalue of the evaluated qubit operator

In [2]:
from qiskit_nature.properties import Property, GroupedProperty

The `qiskit_nature.properties` module provides two classes:
1. `Property`: this is the basic interface. It requires only a `name` and an `interpret` method to be implemented.
2. `GroupedProperty`: this class is an implementation of the [Composite pattern](https://en.wikipedia.org/wiki/Composite_pattern) which allows you to _group_ multiple properties into one.
> Grouped properties must have unique `name` attributes!

## Second Quantization Properties

At the time of writing, Qiskit Nature ships with a single variant of properties: the `SecondQuantizedProperty` objects.
This sub-type adds one additional requirement because any `SecondQuantizedProperty`
* **must** implement a `second_q_ops` method which constructs a (list of) `SecondQuantizedOp`s.

The `qiskit_nature.properties.second_quantization` module is further divided into `electronic` and `vibrational` modules (similar to other modules of Qiskit Nature).
Let us dive into the `electronic` sub-module first.

### Electronic Second Quantization Properties

Out-of-the-box Qiskit Nature ships the following electronic properties:

In [3]:
from qiskit_nature.properties.second_quantization.electronic import (
    ElectronicEnergy,
    ElectronicDipoleMoment,
    ParticleNumber,
    AngularMomentum,
    Magnetization,
)

/home/oss/Files/Qiskit/.direnv/python-3.9.6/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


`ElectronicEnergy` is arguably the most important property because it contains the _Hamiltonian_ representing the electronic structure problem whose eigenvalues we are interested in when solving an `ElectronicStructureProblem`.
The way in which it stores this Hamiltonian is, just like the rest of the framework, highly modular. The initializer of the `ElectronicEnergy` has a single required argument, `electronic_integrals`, which is a `List[ElectronicIntegrals]`. Now you may ask: "what are those?", but bear with me.


#### `ElectronicIntegrals`

In [4]:
from qiskit_nature.properties.second_quantization.electronic.integrals import (
    ElectronicIntegrals,
    OneBodyElectronicIntegrals,
    TwoBodyElectronicIntegrals,
    IntegralProperty,
)

The `ElectronicIntegrals` are a container class for _n-body_ interactions in a given basis, which can be any of the following:

In [5]:
from qiskit_nature.properties.second_quantization.electronic.bases import ElectronicBasis

In [6]:
list(ElectronicBasis)

[<ElectronicBasis.AO: 'atomic'>,
 <ElectronicBasis.MO: 'molecular'>,
 <ElectronicBasis.SO: 'spin'>]

Let us take the `OneBodyElectronicIntegrals` as an example. As the name suggests, this container is for 1-body interaction integrals. You can construct an instance of it like so:

In [7]:
one_body_ints = OneBodyElectronicIntegrals(
    ElectronicBasis.MO,
    (
        np.eye(2),
        2 * np.eye(2),
    ),
)
print(one_body_ints)

(MO) 1-Body Terms:
	Alpha
	[0, 0] = 1.0
	[1, 1] = 1.0
	Beta
	[0, 0] = 2.0
	[1, 1] = 2.0


The second argument requires a little further explanation:

1. In the case of the `AO` or `MO` basis, this argument **must** be a pair of numpy arrays, where the first and second one are the alpha- and beta-spin integrals, respectively.
> The second argument may be `None`, in which case the alpha-spin integrals are used instead.

2. In the case of the `SO` basis, this argument **must** be a single numpy array, storing the alpha- and beta-spin integrals in blocked order, i.e. like so:
```python
spin_basis = np.block([[alpha_spin, zeros], [zeros, beta_spin]])
```

The `TwoBodyElectronicIntegrals` work pretty much the same except that they contain all possible spin-combinations in the tuple of numpy arrays. For example:

In [8]:
two_body_ints = TwoBodyElectronicIntegrals(
    ElectronicBasis.MO,
    (
        np.arange(1, 17).reshape((2, 2, 2, 2)),
        np.arange(16, 32).reshape((2, 2, 2, 2)),
        np.arange(-16, 0).reshape((2, 2, 2, 2)),
        None,
    ),
)
print(two_body_ints)

(MO) 2-Body Terms:
	Alpha-Alpha
	[0, 0, 0, 0] = 1
	[0, 0, 0, 1] = 2
	[0, 0, 1, 0] = 3
	[0, 0, 1, 1] = 4
	[0, 1, 0, 0] = 5
	[0, 1, 0, 1] = 6
	[0, 1, 1, 0] = 7
	[0, 1, 1, 1] = 8
	[1, 0, 0, 0] = 9
	[1, 0, 0, 1] = 10
	[1, 0, 1, 0] = 11
	[1, 0, 1, 1] = 12
	[1, 1, 0, 0] = 13
	[1, 1, 0, 1] = 14
	[1, 1, 1, 0] = 15
	[1, 1, 1, 1] = 16
	Beta-Alpha
	[0, 0, 0, 0] = 16
	[0, 0, 0, 1] = 17
	[0, 0, 1, 0] = 18
	[0, 0, 1, 1] = 19
	[0, 1, 0, 0] = 20
	[0, 1, 0, 1] = 21
	[0, 1, 1, 0] = 22
	[0, 1, 1, 1] = 23
	[1, 0, 0, 0] = 24
	[1, 0, 0, 1] = 25
	[1, 0, 1, 0] = 26
	[1, 0, 1, 1] = 27
	[1, 1, 0, 0] = 28
	[1, 1, 0, 1] = 29
	[1, 1, 1, 0] = 30
	[1, 1, 1, 1] = 31
	Beta-Beta
	[0, 0, 0, 0] = -16
	[0, 0, 0, 1] = -15
	[0, 0, 1, 0] = -14
	[0, 0, 1, 1] = -13
	[0, 1, 0, 0] = -12
	[0, 1, 0, 1] = -11
	[0, 1, 1, 0] = -10
	[0, 1, 1, 1] = -9
	[1, 0, 0, 0] = -8
	[1, 0, 0, 1] = -7
	[1, 0, 1, 0] = -6
	[1, 0, 1, 1] = -5
	[1, 1, 0, 0] = -4
	[1, 1, 0, 1] = -3
	[1, 1, 1, 0] = -2
	[1, 1, 1, 1] = -1
	Alpha-Beta
	[0, 0, 0, 0] = 16
	[0,

We should take note of a few observations:
* the numpy arrays shall be ordered as `("alpha-alpha", "beta-alpha", "beta-beta", "alpha-beta")`
* the `alpha-alpha` matrix may **not** be `None`
* if the `alpha-beta` matrix is `None`, but `beta-alpha` is not, its transpose will be used (like above)
* in any other case, matrices which are `None` will be filled with `alpha-alpha` matrix

* in the `SO` basis case, a single numpy array must be specified. Refer to `TwoBodyElectronicIntegrals.to_spin()` for its exact formatting.

#### `ElectronicEnergy`

Now we are ready to construct an `ElectronicEnergy` instance:

In [9]:
electronic_energy = ElectronicEnergy(
    [one_body_ints, two_body_ints],
)
print(electronic_energy)

ElectronicEnergy
	(MO) 1-Body Terms:
		Alpha
		[0, 0] = 1.0
		[1, 1] = 1.0
		Beta
		[0, 0] = 2.0
		[1, 1] = 2.0
	(MO) 2-Body Terms:
		Alpha-Alpha
		[0, 0, 0, 0] = 1
		[0, 0, 0, 1] = 2
		[0, 0, 1, 0] = 3
		[0, 0, 1, 1] = 4
		[0, 1, 0, 0] = 5
		[0, 1, 0, 1] = 6
		[0, 1, 1, 0] = 7
		[0, 1, 1, 1] = 8
		[1, 0, 0, 0] = 9
		[1, 0, 0, 1] = 10
		[1, 0, 1, 0] = 11
		[1, 0, 1, 1] = 12
		[1, 1, 0, 0] = 13
		[1, 1, 0, 1] = 14
		[1, 1, 1, 0] = 15
		[1, 1, 1, 1] = 16
		Beta-Alpha
		[0, 0, 0, 0] = 16
		[0, 0, 0, 1] = 17
		[0, 0, 1, 0] = 18
		[0, 0, 1, 1] = 19
		[0, 1, 0, 0] = 20
		[0, 1, 0, 1] = 21
		[0, 1, 1, 0] = 22
		[0, 1, 1, 1] = 23
		[1, 0, 0, 0] = 24
		[1, 0, 0, 1] = 25
		[1, 0, 1, 0] = 26
		[1, 0, 1, 1] = 27
		[1, 1, 0, 0] = 28
		[1, 1, 0, 1] = 29
		[1, 1, 1, 0] = 30
		[1, 1, 1, 1] = 31
		Beta-Beta
		[0, 0, 0, 0] = -16
		[0, 0, 0, 1] = -15
		[0, 0, 1, 0] = -14
		[0, 0, 1, 1] = -13
		[0, 1, 0, 0] = -12
		[0, 1, 0, 1] = -11
		[0, 1, 1, 0] = -10
		[0, 1, 1, 1] = -9
		[1, 0, 0, 0] = -8
		[1, 0, 0,

This property can now be used to construct a `SecondQuantizedOp` (which can then be mapped to a `QubitOperator`):

In [10]:
hamiltonian = electronic_energy.second_q_ops()[0]  # here, output length is 1
print(hamiltonian)

  +-+- * (22+0j)
+ +--+ * (-26+0j)
+ +-IN * (30+0j)
+ +-NI * (18+0j)
+ -++- * (-21+0j)
+ -+-+ * (25+0j)
+ -+IN * (-29+0j)
+ -+NI * (-17+0j)
+ IIIN * (2+0j)
+ IINI * (2+0j)
+ IN+- * (23+0j)
+ IN-+ * (-27+0j)
+ INII * (1+0j)
+ ININ * (31+0j)
+ INNI * (19+0j)
+ NI+- * (20+0j)
+ NI-+ * (-24+0j)
+ NIII * (1+0j)
+ NIIN * (28+0j)
+ NINI * (16+0j)


An additional benefit which we gain from the `Property` framework, is that the result interpretation of a computed eigenvalue can be handled by each property itself. This results in nice and logically consistent classes because the result gets interpreted in the same context where the raw data is available.

In [11]:
from qiskit_nature.results import ElectronicStructureResult

# some dummy result
result = ElectronicStructureResult()
result.eigenenergies = np.asarray([-1])
result.computed_energies = np.asarray([-1])


# now, let's interpret it
electronic_energy.interpret(result)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1
  - computed part:      -1


While this particular example is not yet very impressive, wait until we use more properties at once.

#### `ParticleNumber`

The `ParticleNumber` property also takes a special place among the builtin properties because it serves a dual purpose of:
* storing the number of particles in the electronic system
* providing the operators to evaluate the number of particles for a given eigensolution of the problem
Therefore, this property is required if you want to use additional functionality like the `ActiveSpaceTransformer` or the `ElectronicStructureProblem.default_filter_criterion`.

In [12]:
particle_number = ParticleNumber(
    num_spin_orbitals = 4,
    num_particles = (1, 1), 
)
print(particle_number)

ParticleNumber:
	4 SOs
	1 alpha electrons: [1. 0.]
	1 beta electrons: [1. 0.]


#### `GroupedProperty`

Rather than iterating all of the other properties one by one, let us simply consider a group of properties as provided by any `ElectronicStructureDriver` from the `qiskit_nature.drivers.second_quantization` module.

In [13]:
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver

In [14]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto3g")
driver_result = driver.run()

In [15]:
print(driver_result)

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 1.7.6
		Config:
			atom=H 0 0 0; H 0 0 0.735
			unit=Angstrom
			charge=0
			spin=0
			basis=sto3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.5483020229014736
		[0, 1] = -1.2183273138546826
		[1, 0] = 0.548302022901473
		[1, 1] = 1.2183273138546828
		Beta coefficients:
		[0, 0] = 0.5483020229014736
		[0, 1] = -1.2183273138546826
		[1, 0] = 0.548302022901473
		[1, 1] = 1.2183273138546828
	ParticleNumber:
		4 SOs
		1 alpha electrons: [1. 0.]
		1 beta electrons: [1. 0.]
	ElectronicEnergy
		(AO) 1-Body Terms:
			Alpha
			[0, 0] = -1.1242175791954514
			[0, 1] = -0.9652573993472758
			[1, 0] = -0.9652573993472758
			[1, 1] = -1.1242175791954512
			Beta
			[0, 0] = -1.1242175791954514
			[0, 1] = -0.9652573993472758
			[1, 0] = -0.9652573993472758
			[1, 1] =

In [17]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright